In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np
import json
import os
from itertools import cycle
from datetime import datetime
from typing import Iterable

In [ ]:
# Setup Color generator
cycol = cycle([
    "blue",
    "green",
    "red",
    "orange",
    "magenta",
    ])

In [ ]:
# Goal Timestamp Creation
def provide_goal_timestamp(timestamp: Iterable[datetime]) -> Iterable[datetime]:
    return [timestamp[0] + pd.to_timedelta("30 sec")]

In [ ]:
# Get actual Smartwatch data in folder
def get_hkquantity_hr_file() -> str:
    for filename in os.listdir("."):
        if filename.startswith("HKQuantityTypeIdentifierHeartRate_"):
            return filename

In [ ]:
def rr_interval_sdnn(df: pd.DataFrame, nn_value: str, rr_column_name: str) -> pd.Series:
    tmp_data = []
    tmp_index = []
    for element in df.itertuples():
        tmp_data.append(
            df[element.Index - pd.to_timedelta(nn_value) : element.Index][rr_column_name].std()
            )
        tmp_index.append(element.Index)
    return pd.Series(data=tmp_data, index=tmp_index)

In [ ]:
# Read Metadata
with open("metadata.json", mode="r") as file:
    metadata = json.load(file)

In [ ]:
# Liveticker Daten
df_liveticker = pd.read_csv("liveticker.csv")
df_liveticker["timestamp"] = pd.to_datetime(df_liveticker["timestamp"])
df_liveticker = df_liveticker.set_index("timestamp")

In [ ]:
# Heartrate Daten
df_heartrate = pd.read_csv(get_hkquantity_hr_file(), skiprows=1)
timestamp = pd.to_datetime(df_heartrate["startDate"])
df_heartrate.insert(0, "timestamp", timestamp)
df_heartrate = df_heartrate.set_index("timestamp")
df_heartrate = df_heartrate[["value", "HKMetadataKeyHeartRateMotionContext"]]

In [ ]:
# Calculate Feature and set limits

# RR Interval
rr_interval = 60000 / df_heartrate["value"]
df_heartrate.insert(1, "rr_interval", rr_interval)
# SDNN 1 min
sdnn = rr_interval_sdnn(df_heartrate, "1 min", "rr_interval")
df_heartrate.insert(2, "sdnn", sdnn)
# Set Limits
df_heartrate = df_heartrate[metadata["start_datetime"] : metadata["end_datetime"]]


In [ ]:
# Merge DataFrames
df_merged = pd.merge_asof(
    df_heartrate, df_liveticker,
    left_index=True,
    right_index=True,
    tolerance=pd.to_timedelta("1 min"),
    direction="backward"
    )

In [ ]:
# Prepare Plotting
rr_interval = df_merged.rr_interval.values
df_merged.is_goal.replace({
    True: 200,
    False: 0,
    np.nan: 0},
    inplace=True
    )
df_merged.is_card.replace({
    True: 100,
    False: 0,
    np.nan: 0},
    inplace=True
    )
df_merged.halftime.replace({
    1: 100,
    2: 100,
    np.nan: 0},
    inplace=True
    )
liveticker_value = df_merged["is_goal"] + df_merged["is_card"] + df_merged["halftime"]
df_merged["liveticker_value"] = liveticker_value

In [ ]:
# Plotting Data
x = df_merged.index.values
x = x.reshape(len(x), 1)
rr_interval = df_merged.rr_interval.values
rr_interval = rr_interval.reshape(len(rr_interval), 1)
liveticker_value = df_merged["liveticker_value"].values
liveticker_value = liveticker_value.reshape(len(liveticker_value), 1)
df_goals = df_liveticker[df_liveticker["is_goal"]]

In [ ]:
# Final Plotting
%matplotlib inline
# Set SNS
sns.set()
sns.set_style("whitegrid")
sns.set_context("poster")
# Plot size
plt.figure(figsize=(100, 20), dpi=50)
# Set axis label
plt.xlabel("Zeitstempel")
plt.ylabel("ms")
# Plot RR Interval
plt.plot(
    x,
    rr_interval,
    c=next(cycol),
    label="rr Interval"
    )
# Plot Liveticker Data
plt.plot(
    x,
    liveticker_value,
    c=next(cycol),
    label="Eintrag Liveticker"
    )
# Plot and Mark Break
break_timerange = (
    pd.to_datetime(metadata["break_start"]),
    pd.to_datetime(metadata["break_end"])
    )
break_color = next(cycol)
plt.plot(
    break_timerange,
    (50,50),
    color=break_color,
    #linewidth=8,
    label="Pause"
    )
plt.fill_between(
    break_timerange,
    (df_heartrate["rr_interval"].max(), df_heartrate["rr_interval"].max()),
    color=break_color,
    alpha=0.25
    )
# mark team goals
for team in set(metadata["team_shortname_mapping"].values()):
    team_goals = df_goals[df_goals["relevant_team"] == team].index.values
    team_goals = team_goals.reshape(len(team_goals), 1)
    team_goals = list(map(provide_goal_timestamp, team_goals))
    plt.scatter(
        team_goals,
        [300 for _ in range(len(team_goals))],
        c=next(cycol),
        label=f"Tor {team}"
        )
# plot relvant line
line_high = df_merged[(df_merged.index <= pd.to_datetime(metadata["break_start"])) | (df_merged.index >= pd.to_datetime(metadata["break_end"]))]["rr_interval"].quantile(q=0.10)
plt.plot(
    (min(df_merged.index.values), max(df_merged.index.values)),
    (line_high, line_high),
    color="black",
    alpha=0.5,
    label="Q 10%"
)
# Axis Settings
ax = plt.gca()
ax.xaxis.set_major_formatter(
    mdates.DateFormatter('%d.%m.%Y %H:%M')
    )
plt.xlim(
    pd.to_datetime(metadata["start_datetime"]),
    pd.to_datetime(metadata["end_datetime"])
    )
handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles,
    labels,
    loc="center left"
    )
# Plot
plt.show()


In [ ]:
# Plotting Data
x = df_merged.index.values
x = x.reshape(len(x), 1)
sdnn = df_merged.sdnn.values
sdnn = sdnn.reshape(len(sdnn), 1)
liveticker_value = df_merged["liveticker_value"].values / 10
liveticker_value = liveticker_value.reshape(len(liveticker_value), 1)
df_goals = df_liveticker[df_liveticker["is_goal"]]

In [ ]:
# Final Plotting
%matplotlib inline
# Set SNS
sns.set()
sns.set_style("whitegrid")
sns.set_context("poster")
# Plot size
plt.figure(figsize=(100, 20), dpi=50)
# Set axis label
plt.xlabel("Zeitstempel")
plt.ylabel("ms")
# Plot RR Interval
plt.plot(
    x,
    sdnn,
    c=next(cycol),
    label="SDNN (1 min)"
    )
# Plot Liveticker Data
plt.plot(
    x,
    liveticker_value,
    c=next(cycol),
    label="Eintrag Liveticker"
    )
# Plot and Mark Break
break_timerange = (
    pd.to_datetime(metadata["break_start"]),
    pd.to_datetime(metadata["break_end"])
    )
break_color = next(cycol)
plt.plot(
    break_timerange,
    (50,50),
    color=break_color,
    #linewidth=8,
    label="Pause"
    )
plt.fill_between(
    break_timerange,
    (df_heartrate["sdnn"].max(), df_heartrate["sdnn"].max()),
    color=break_color,
    alpha=0.25
    )
# mark team goals
for team in set(metadata["team_shortname_mapping"].values()):
    team_goals = df_goals[df_goals["relevant_team"] == team].index.values
    team_goals = team_goals.reshape(len(team_goals), 1)
    team_goals = list(map(provide_goal_timestamp, team_goals))
    plt.scatter(
        team_goals,
        [30 for _ in range(len(team_goals))],
        c=next(cycol),
        label=f"Tor {team}"
        )
# plot relvant line
line_high = df_merged[(df_merged.index <= pd.to_datetime(metadata["break_start"])) | (df_merged.index >= pd.to_datetime(metadata["break_end"]))]["sdnn"].quantile(q=0.75)
plt.plot(
    (min(df_merged.index.values), max(df_merged.index.values)),
    (line_high, line_high),
    color="black",
    alpha=0.5,
    label="Q 75%"
)
# Axis Settings
ax = plt.gca()
ax.xaxis.set_major_formatter(
    mdates.DateFormatter('%d.%m.%Y %H:%M')
    )
plt.xlim(
    pd.to_datetime(metadata["start_datetime"]),
    pd.to_datetime(metadata["end_datetime"])
    )
handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles,
    labels,
    loc="upper left"
    )
# Plot
plt.savefig("plot.png")
plt.show()


In [ ]:
df_merged[["rr_interval", "sdnn"]].describe()

In [ ]:
df_merged[(df_merged.index <= pd.to_datetime(metadata["break_start"])) | (df_merged.index >= pd.to_datetime(metadata["break_end"]))].boxplot(column=["sdnn"], rot=0)
